In [ ]:
from qiskit import IBMQ
import qiskit as q
qasm_sim = q.Aer.get_backend('qasm_simulator')                  #simulator backend
statevec_sim = q.Aer.get_backend("statevector_simulator") #state vector simulator backend
backend = qasm_sim

In [ ]:
from matplotlib import style
%matplotlib inline

shots_num = 1000 #numbers of shots in experiment

def run_exp(circuit, bcknd):  
    num_qubits = circuit.num_qubits
    circuit.measure([i for i in range(num_qubits)], [i for i in range(num_qubits)])

    result = q.execute(circuit, backend=bcknd, shots=shots_num).result()
    counts = result.get_counts()
    
    return counts

In [ ]:
def run_statevector(circuit, bcknd): 
    job = q.execute(circuit, backend=bcknd)
    result = job.result()
    statevec = result.get_statevector()
    return statevec

In [ ]:
def my_range(start, end, step):
    while start <= end:
        yield start
        start += step

In [ ]:
#setting payoffs
D = 3;
S = 2;
P = 0;

In [ ]:
from scipy.interpolate import BSpline, make_interp_spline
#function to draw a plot
def draw_plot(plt,x,y,name):
    plt.ylabel('Payoff, $')
    plt.xlabel('Entanglement degree, γ')
    plt.ylim(P, D, 1)

    x_new = np.linspace(0, 1, 50)
    a_BSpline = make_interp_spline(x, y)
    y_new = a_BSpline(x_new)
    plt.plot(x_new, y_new, label =name)
  
    plt.legend(loc='best')
    font = {'family' : 'Times New Roman',
            'weight' : 'regular',
            'size'   : 18}

    plt.rc('font', **font)
    fig = plt.gcf()
    fig.set_size_inches(10, 8)

In [ ]:
from qiskit.circuit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.quantum_info.operators import Operator, Pauli
import cmath, math, numpy as np

def getQC(gamma):
    qbits = QuantumRegister(2)
    clbits = ClassicalRegister(2)
    qc = QuantumCircuit(qbits, clbits)
    
    J = 1/math.sqrt(1+gamma*gamma)*Operator(Operator(Pauli(label='II'))+complex(0,gamma)*Operator(Pauli(label='XX'))) #entangling operator J

    theta = math.pi/2
    phi = 0
    lmbda = 0
  
    qc.unitary(J, [0, 1])
    qc.u3(theta, phi, lmbda, [0])                #Alice's rotation operator
    qc.u3(theta, phi, -math.pi/2 - lmbda, [1])   #Bob's rotation operator
    qc.unitary(J.adjoint(), [0, 1])
    return qc

In [ ]:
import csv
import datetime

Alice_payoff = np.array([]) #Alice's payoff funtion
Bob_payoff = np.array([])   #Bob's payoff funtion
x = np.array([])            #x axis for plot

counts = {'00': 0, '01': 0, '10': 0, '11': 0}

for gamma in my_range(0, 1, 0.1):
    qc = getQC(gamma)
    result = run_exp(qc, backend)        #runs experiment
    counts.update(result)

    Alice_average_payoff = (D * counts['00'] + P * (counts['01'] + counts['10']) + S * counts['11'])/shots_num
    Bob_average_payoff = (S * counts['00'] + P * (counts['01'] + counts['10']) + D * counts['11'])/shots_num  
    Alice_payoff = np.append(Alice_payoff, Alice_average_payoff)
    Bob_payoff = np.append(Bob_payoff, Bob_average_payoff)
    x = np.append(x, gamma)

In [ ]:
import matplotlib.pyplot as plt
#displaying plot
fig = plt.gcf()
draw_plot(plt,x,Alice_payoff,"Alice")
draw_plot(plt,x,Bob_payoff,"Bob")
plt.show()
plt.draw()
#fig.savefig("plot_"+backend.name()+'_'+now.strftime("%Y-%m-%d %H-%M-%S")+".png", dpi=100)

plot_histogram([counts], legend=['output'])

In [ ]:
from qiskit.tools.visualization import plot_bloch_multivector
#show state vector of the maximally entangled state
statevec = run_statevector(getQC(1.0), statevec_sim)
plot_bloch_multivector(statevec)